In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import minmax_scale

# Get weather dataset

In [3]:
weather_dataset = pd.read_csv('weather_wwo_manila_2015.csv', skipinitialspace=True, encoding='cp1252')
cols = ["tempC", "windspeedKmph", "cond", "precipMM", "humidity", "visibility", "pressure", "cloudcover", "heatIndexC", "dewPointC", "windChillC", "windGustKmph", "feelsLikeC"]
weather_dataset = weather_dataset[cols]

# Correlate traffic and weather dataset

In [4]:
roads = ["A. Maceda", "Anda Circle", "Antipolo", "Bluementritt", "Buendia", "Edsa Extension", "Finance Road", "Gov. Forbes - Lacson", "Lerma", "Magsaysay Ave", "P.Noval", "Pablo Ocampo", "Pedro Gil", "Quezon Ave.", "Quirino", "Rajah Sulayman", "Taft Ave.", "U.N. Avenue", "Vicente Cruz"]

In [5]:
for road in roads:
    traffic_dataset = pd.read_csv('mmda_2015_transformed/mmda_' + road + '_2015_transformed.csv', skipinitialspace=True, encoding='cp1252')
    traffic_dataset = traffic_dataset[['statusN', 'statusS']]
    
    # merge traffic and weather dataset
    dataset = pd.concat([traffic_dataset, weather_dataset], axis=1, join='inner')
    
    # correlate
    corr = dataset.corr(method='spearman')
    corr.to_csv('corr_mmda_wwo_2015/corr_results_mmda_wwo_' + road + '_2015.csv')